#### Etapa preliminar - Equação de advecção pura 

Desenvolver um sistema para resolução da equação de advecção pura, para melhor compreensão das ferramentas necessárias e do método 

Equação avecção pura 1-D:

\begin{equation}
    \frac{dC}{dt} + c\frac{dC}{dx} = 0
\end{equation}

Solução analítica:

\begin{equation}
    C(x,t) = C_{0}(x - ct)
\end{equation}

Condição inicial:

\begin{equation}
    C(x,t=0) = C_{0}(t)
\end{equation}

Condição de contorno:

\begin{equation}
    C(x=x_{inicial},t) = 0
\end{equation}

\begin{equation}
    C(x=x_{final}, t) = 0
\end{equation}






In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import division
from sympy import *
import pandas as pd


In [ ]:
# Teste de estabilidade - número de Courant = velocidade *(dt/dx) <= 1

# dt - passo no tempo; tmax - tempo máximo de análise; x0 - posição inicial; n - número de passos de tempo dentro do tmax; u - velocidade
def courant(dt, tmax,x0, u):

    n = round(tmax/dt)

    dx = (tmax - x0) /n

    C = u * (dt/dx)

    if C>1:
        return print('Unstable, reduce dt', dx )
    if C <= 1:
        return print('Stable', dx)

courant(0.0001,100.0,0., 1.0)

In [ ]:
# Resolução analítica
# Malha
x0 =0
L = 100.0
t_max = 100.0
t0 = 0

dx = 0.1
dt = 0.1

x = np.arange(x0, L, dx)
t = np.arange(0, t_max, dt)


# Constantes do sistema
u = 0.5

# Condição inicial 

C_0 = 0

# Condição de contorno

C_x0 = 0
C_xn = 0

C_adveccao_results = np.empty_like(t)

In [ ]:
contmax = len(t)
       
while t0 < t_max:
    for i in range (len(t)):
             
        C_adveccao = C_0 * (x[i] - u*t[i])
        
        if i == len(t-1):
            C_adveccao = C_xn
            C_adveccao_results[i] = C_adveccao  # não está funcionando entender o pq ,mas a equação foi resolvida

        else:
            C_adveccao_results[i] = C_adveccao 
        
       
        t0 = t0 + dt
    


plt.plot(t, C_adveccao_results)


In [ ]:
# Solução numérica

# TDMA solver

# Malha
x0 =0
L = 100.0
nx = 100.0 # number of nodes
t_max = 100.0
nt = 100.0
t0 = 0

dx = L/ nx
dt = t_max / nt

x = np.arange(x0, L, dx)
t = np.arange(0, t_max, dt)


# Constantes do sistema
u = 0.5

# Condição inicial 

C_0 = 10

# Condição de contorno

C_x0 = 0
C_xn = 0

n = len(t) # numeros de elementos nas diagonais

# lista/array vazio para cada elemento 

D = np.zeros(n) # valores da diagonal principal
beta = np.zeros(n) # valores da diagonal inferior - lembrar que beta é 0 na primeira iteração
alfa = np.zeros(n) # valores da diagonal superior - lembrar que alfa é 0 na ultima iteração
k = np.zeros(n) # valores da constante à direita - solução

X = np.zeros(n)

def tdma(a, b, c, d):
    """Solution of a linear system of algebraic equations with a
        tri-diagonal matrix of coefficients using the Thomas-algorithm.

    Args:
        a(array): an array containing lower diagonal (a[0] is not used)
        b(array): an array containing main diagonal 
        c(array): an array containing lower diagonal (c[-1] is not used)
        d(array): right hand side of the system
    Returns:
        x(array): solution array of the system
    
    """
    
    n = len(b)
    x = np.zeros(n)
    
    # elimination:
    
    for k in range(1,n):
        q = a[k]/b[k-1]
        b[k] = b[k] - c[k-1]*q
        d[k] = d[k] - d[k-1]*q
    
    # backsubstitution:
    
    q = d[n-1]/b[n-1]
    x[n-1] = q
    
    for k in range(n-2,-1,-1):
        q = (d[k]-c[k]*q)/b[k]
        x[k] = q
    
    
    return x

#### Trabalho 3

Desenvolver um modelo numérico 1D para estudar o transporte de sal. O modelo
desenvolvido deve ser aplicado em um canal com geometrias:

    Extensão de 100900,0 m

    1000,0 m de largura 

    Seção transversal retangular

    4,0 m de profundidade

    Na entrada do canal há uma contração localizada à 400,0 m da seção 1 (L1). A contração possui 500,0 m de extensão e largura 

#### Solução analítica

A equação de advecção-difusão 1D:


\begin{equation}
    \frac{dC}{dt} + c\frac{dC}{dx} = D*\frac{d^{2}C}{dx^{2}}
\end{equation}

Com as seguintes condições iniciais e de contorno:

\begin{equation}
    C(x,t=0) = C_{0}(t)
\end{equation}

Condição de contorno:

\begin{equation}
c(0, t)=\left\{\begin{array}{lc}c_{0} & 0 < t < t_{0} \\
0 & t>t_{o}\end{array}\right.
\end{equation}

\begin{equation}
\frac{\partial c}{\partial x}(\infty, t)=0
\end{equation}




Solução :

\begin{equation}
\begin{aligned}
&c(x, t)=\\ &\left\{\begin{array}{l}\frac{\gamma}{\mu}+\left(C_{i}-\frac{\gamma}{\mu}\right)*A(x, t)+\left(C_{0}-\frac{\gamma}{\mu}\right)*B(x, t) & 0 < t < t_{0} \\
\frac{\gamma}{\mu}+\left(C_{i}-\frac{\gamma}{\mu}\right) * A(x, t)+\left(C_{0}-\frac{\gamma}{\mu}\right)* B(x, t)-C_{0} *B\left(x, t-t_{0}\right) & t>t_{0}\end{array}\right.
\end{aligned}
\end{equation}

Onde

A(x,t):

\begin{equation}
\begin{aligned}
A(x, t)=\exp (-\mu t / R)\left\{1-\frac{1}{2} \operatorname{erfc}\left[\frac{\mathrm{Rx}-\mathrm{v} t}{2(\mathrm{DRt})^{1 / 2}}\right]\right.\\
&-\frac{1}{2} \exp (\mathrm{vx} / \mathrm{D}) \operatorname{erfc} \left[\frac{\mathrm{Rx}+\mathrm{v} t}{2(\mathrm{DRt})^{1 / 2}}\right]

\end{aligned}
\end{equation}

B(x,t):


\begin{equation}
\begin{aligned}
B(x, t) &=\frac{1}{2} \exp \left[\frac{(v-u) x}{2 D}\right] \operatorname{erfc}\left[\frac{R x-u t}{2(D R t)^{1 / 2}}\right] \\
&+\frac{1}{2} \exp \left[\frac{(v+u) x}{2 D}\right] \operatorname{erfc}\left[\frac{R x+u t}{2(D R t)^{1 / 2}}\right]
\end{aligned}
\end{equation}

\begin{equation}
u=v\left(1+\frac{4 \mu \mathrm{D}}{v^{2}}\right)^{1 / 2}
\end{equation}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.function_base import copy

n = 81
L = 1.0
x = np.linspace(0,L,n)
dx = L/(n-1)
assert(dx == x[1]-x[0])

w = 2.0*np.pi
mu = 0.05
c = 1.0 # wave speed
u0 = lambda x: np.sin(w*x)
uexact = lambda x,t: u0(x-c*t) * np.exp(-mu*w*w*t)

dt = 0.001
tend = 1.0
t = 0

cfl = c*dt/2.0/dx
nfl = mu*dt/dx/dx
sol = []
sol.append(u0)

while t < tend:
    un = sol[-1]
    unew = un.copy()
    unew[1:-1] = un[1:-1] - cfl * (un[2:] - un[:-2]) + nfl*(un[2:] - 2.0*un[1:-1] + un[:-2])
    unew[-1] = un[-1] - cfl * (un[1] - un[-2]) + nfl*(un[-2] - 2.0*un[-1] + un[1])
    unew[0] = unew[-1]
    sol.append(unew)
    t += dt

####  Solução numérica 

Para U 0 :

\begin{equation}
C_{j+1}^{n+1}*(\alpha - \beta) + C_{j}^{n+1}* (1- \alpha +2*\beta) + C_{j-1}^{n+1} *(-\beta) = C_{j}^{n} 
\end{equation}

Para U 0 :

\begin{equation}
C_{j+1}^{n+1}*(- \beta) + C_{j}^{n+1}* (1 + \alpha +2*\beta) + C_{j-1}^{n+1} *(-\alpha -\beta) = C_{j}^{n} 
\end{equation}

In [ ]:
#===============================================================
# Some libraries
#===============================================================

import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rc
rc('font', family='serif')
rc('lines', linewidth=1.5)
rc('font', size=16)
plt.rc('legend',**{'fontsize':14})

#===============================================================
# Define parameters
#===============================================================

D     = 0.01                          # Diffusivity
T1    = - beta                        # Diagonal inferior 
T2    = 1 - alfa + 2*beta             # Diagonal meio
T3    = alfa - beta                   # Diagonal superior
Nx    = 101                           # Number of grid points
xmax  = 10.                           # Domain limit to the right
xmin  = 0.                            # Domain limit to the left
dx    = (xmax-xmin)/(Nx-1)            # Mesh size
x     = np.linspace(xmin,xmax,Nx)     # Discretized mesh
t_end = 5.                            # Final time
dt    = 0.001                         # Time step 
Nt    = int(t_end/dt)                 # Number of iterations
t     = np.linspace(0.,t_end,Nt+1)    # Time vector
c     = 0.5                           # Celeridade do fluxo
alfa  = c*dt/dx                       # Stability condition - Udt/dx
beta  = D*dt/(dx**2)                  # Stability condition - Ddt/dx^2
U     = zeros((2,Nt+1), float)        # Matriz resultado

#===============================================================
# Initial condition
#===============================================================

# def CI(x):                      # define a condição inicial
#   if xmin <= x <= xmax:
#     return 2.0*x*(1.0-x)          
#   else:
#     return 0.0

# for i in range(Nx+1):           # monta a condição inicial
#   xi = i*dx
#   U[0,i] = CI(xi)

U[0,:] = np.exp(-0.5*(x/0.4)**2)  # Inserir o que define a C.I

#===============================================================
# Stability check
#===============================================================

courant = c*dt/dx                 # Número de Courant <= 1
pe = c * dx /D                    # Núemro de pèclet <= 2
neumann = D *dt/dx**2             # Von Neumann <= 1/2

#===============================================================
# Temporal loop
#===============================================================

for n in range (0,Nt):
    
  for i in range (1,Nx-1):
      U[n+1,i] = U[n,i] + sigma*(U[n,i+1]-2*U[n,i]+U[n,i-1]); # Interior points
  U[n+1,0] = 0;                                               # BC left
  U[n+1,-1] = 0;                                              # BC right
  
  # # Plot solution
  # if (n==0): fig, ax = plt.subplots(figsize=(5.5,4))
  # plt.clf()
  # plt.plot(x,U[n+1,:])
  # plt.scatter(x,U[0,:], marker='o', facecolors='white', color='k')
  # plt.gca().legend(('Numerical result ($\sigma$='+str(sigma)+')','Initial condition'))
  # plt.axis([xmin, xmax, 0, 1.4])
  # plt.title('t='+str(round(dt*(n+1),3)),fontsize=16)
  # plt.xlabel('x',fontsize=18)
  # plt.ylabel('u',fontsize=18)
  # plt.subplots_adjust(left=0.2)
  # plt.subplots_adjust(bottom=0.18)
  # plt.draw()
  # plt.pause(0.001)

In [14]:
from numpy import zeros 
u = zeros((2,nx+1),float) 
u

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [24]:
old = False
new = True
D = 2.0 # celeridade da onda
Fon = D*dt/((dx)**2) # número de Fourier
print("Fo = %10.6f" % Fon)
for n in range(nt): # loop no tempo
  print(n)
  for i in range(1,nx): # loop no espaço
    u[new,i] = u[old,i] + Fon*(u[old,i+1] - 2*u[old,i] + u[old,i-1])
  u[new,0] = 0.0 # condição de contorno, x = 0
  u[new,nx] = 0.0 # condição de contorno, x = 1
  u[new].tofile(fou) # imprime uma linha com os novos dados
  (old,new) = (new,old) # troca os índices

array([0.  , 0.18, 0.32, ..., 0.  , 0.  , 0.  ])

In [ ]:
################################################################
#
# Class 02: 1D-Diffusion problem
# Objective: Solve equation using the Forward-Time Central-Space scheme
# Author: P. S. Volpiani
# Date: 01/08/2020
#
################################################################

#===============================================================
# Some libraries
#===============================================================

import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rc
rc('font', family='serif')
rc('lines', linewidth=1.5)
rc('font', size=16)
plt.rc('legend',**{'fontsize':14})

#===============================================================
# Define parameters
#===============================================================

alpha = 0.01                      # Diffusivity
sigma = 0.4                       # Stability condition
Nx = 101;                         # Number of grid points
xmax = 2.;                        # Domain limit to the right
xmin = -2.;                       # Domain limit to the left
dx = (xmax-xmin)/(Nx-1)           # Mesh size
x = np.linspace(xmin,xmax,Nx)     # Discretized mesh
dt = sigma*dx**2/alpha            # Time step
t_end = 5.                        # Final time
Nt = int(t_end/dt)                # Number of iterations
t = np.linspace(0.,t_end,Nt+1)    # Time vector
U = np.zeros((Nt+1,Nx))           # u^n_i
U[0,:] = np.exp(-0.5*(x/0.4)**2)  # Initial solution


#===============================================================
# Temporal loop
#===============================================================

for n in range (0,Nt):
  
  sigma = alpha*dt/(dx*dx)
  
  for i in range (1,Nx-1):
      U[n+1,i] = U[n,i] + sigma*(U[n,i+1]-2*U[n,i]+U[n,i-1]); # Interior points
  U[n+1,0] = 0;                                               # BC left
  U[n+1,-1] = 0;                                              # BC right
  
  # Plot solution
  if (n==0): fig, ax = plt.subplots(figsize=(5.5,4))
  plt.clf()
  plt.plot(x,U[n+1,:])
  plt.scatter(x,U[0,:], marker='o', facecolors='white', color='k')
  plt.gca().legend(('Numerical result ($\sigma$='+str(sigma)+')','Initial condition'))
  plt.axis([xmin, xmax, 0, 1.4])
  plt.title('t='+str(round(dt*(n+1),3)),fontsize=16)
  plt.xlabel('x',fontsize=18)
  plt.ylabel('u',fontsize=18)
  plt.subplots_adjust(left=0.2)
  plt.subplots_adjust(bottom=0.18)
  plt.draw()
  plt.pause(0.001)

plt.show()
#fig.savefig("figure.pdf", dpi=300)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import mpl_toolkits.mplot3d
import scipy.sparse as sp
import scipy.sparse.linalg 
import scipy.linalg as la
# import dolfin
# import mshr

Estrutura da matriz A*u = b

In [6]:
N = 5                                                       # Number of interior points
u0, u1 = 0, 0                                               # Boundary conditions
xmax = 1                                                    # Length of the system
dx = xmax / (N + 1)                                         # x step size
D     = 0.01                                                # Diffusivity
c     = 0.5                                                 # Celeridade do fluxo
alfa  = c*dt/dx                                             # Stability condition - Udt/dx
beta  = D*dt/(dx**2)                                        # Stability condition - Ddt/dx^2
T1    = - beta                                              # Diagonal inferior 
T2    = 1 - alfa + 2*beta                                   # Diagonal meio
T3    = alfa - beta                                         # Diagonal superior
A = (np.eye(N, k=-1)*T1 + np.eye(N)*T2 + np.eye(N, k=1)*T3) # Matrix 2D para as iterações
b = np.ones(N)                                              # Matriz resultado
b[0] = 0                                                    # Lower boundary condition - C(x=0, t) = 0
b[N-1] = 0                                                  # Upper boundary condition - C(x=xmax,t) = 0
ic = 0                                                      # Initial condition - C(x,0) = 0
u = la.solve(A, b)                                          # Matriz de interesse - resposta
x = np.linspace(0., xmax, N+2)                              # pontos da malha discretizados incluindo os c/ C.C
U = np.hstack([[u0], u, [u1]])                              # Matriz final com os valores das C.C e pontos internos
fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(x, U)
ax.plot(x[1:-1], u, 'ks')
ax.set_xlim(0, 1)
ax.set_xlabel(r"$x$", fontsize=18)
ax.set_ylabel(r"$u(x)$", fontsize=18)



NameError: name 'dt' is not defined